# Assignment - Artificial Intelligence 3

## Assignment Background

As testers you don't have much time, so please feel free to just attempt what you can. A difficult part is gauging the difficulty of an assignment to not bee too easy, too difficult, or too long.

The aim of this assignment is to recreate the workflow of the image section of this lesson, using a different dataset. The dataset will be the Pneumonia dataset that was used in the previous lessons assignment.

Given the time to train and optimise a model, we have trained a CNN that can be found in the [data](../data/pneumonia_model.pth) folder. The architecture has been provided for you to load in the model and had an accuracy of 85% on the test set. 

Your task is to investigate how the model is making correct predictions (has pneumonia) and why it is making incorrect predictions (false negatives).

## Assignment Questions:

## Question 1: 

**Task 1:**
- Download the PneumoniaMNIST
- Load in the validation dataset into a data loader
- Initialise the `PneumoniaCNN` model and update it with the model saved parameters at `../data/pneumonia_model.pth`

The code for most of this has already been given to you.

In [ ]:
import torch
import torch.nn as nn

import warnings
warnings.filterwarnings("ignore") # ignore warnings

# Import the MNIST dataset
from torchvision import transforms

# import medmnist, the medical image dataset
import medmnist
from medmnist import INFO, Evaluator

import matplotlib.pyplot as plt
import numpy as np

# fix the numpy and torch seeds for reproducibility:
SEED = 999
np.random.seed(SEED)
torch.manual_seed(SEED);

In [ ]:
data_flag = 'pneumoniamnist'

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

print(f"The learning task is {task}")
print(f"The number of channels is {n_channels}")
print(f"The number of classes is {n_classes}")

download = True
BATCH_SIZE = 128

DataClass = getattr(medmnist, info['python_class'])

# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5]) # normalise the data with standard values
])

# Validation dataset, do not use this except to check the final score
val_loader = torch.utils.data.DataLoader(
    DataClass(split='val', 
                transform=data_transform, 
                download=download, 
                root='data'),
    batch_size=2*BATCH_SIZE, shuffle=True)

In [ ]:
class PneumoniaCNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=2):
        super().__init__()
        
        # First convolutional block
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),  # 28x28 -> 28x28
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 28x28 -> 14x14
        )
        
        # Second convolutional block
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 14x14 -> 14x14
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 14x14 -> 7x7
        )
        
        # Third convolutional block
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # 7x7 -> 7x7
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 7x7 -> 3x3
        )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.fc(x)
        return x

In [ ]:
# Initalise and load the model here

if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device(f"cuda:0")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'Device name: {torch.cuda.get_device_name(0)}')
else:
    print("No GPU detected! Falling back to CPU")
    # If the GPU is not available, use the CPU
    device = torch.device("cpu")
    

# then define the model (architecture above) and move it to the device:
model = PneumoniaCNN()

# load the trained model that we saved in the last lesson, instead of having to train it again:
model.load_state_dict(torch.load('./data/pneumonia_model.pth'))
model = model.to(device)

**Task 2:**
- Select 6 images from the validation set that are correctly predicted by the model.
- Select 6 images from the validation set that are incorrectly predicted by the model (false negatives).
- In two seperate plots, display the images and their predicted labels.

In [ ]:
test_images = []
test_labels = []
for images, labels in val_loader:
    test_images.append(images)
    test_labels.append(labels)
test_images = torch.cat(test_images, dim=0)
test_labels = torch.cat(test_labels, dim=0)

real_labels = {
            0: "adipose",
            1: "background",
            2: "debris",
            3: "lymphocytes",
            4: "mucus",
            5: "smooth muscle",
            6: "normal colon mucosa",
            7: "cancer-associated stroma",
            8: "crc adenocarcinoma epithelium"
        }

In [ ]:
# obtain one batch of test images
dataiter = iter(val_loader)
images, labels = next(dataiter)
images = images.numpy()

# move model inputs to cuda, if GPU available
images = torch.from_numpy(images)
images = images.to(device)
labels = labels.to(device)

# get sample outputs
model.eval() # set model to evaluation mode
output = model(images)

# convert output probabilities to predicted class
_, preds = torch.max(output, 1)

In [ ]:
correct_indices = []
for i in range(len(labels)):
    if preds[i] == labels[i]:
        correct_indices.append(i)

# shuffle the correct indices list:
np.random.shuffle(correct_indices)
correct_indices = correct_indices[:5]

correct_indices

In [ ]:
# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(18, 6))

for c, idx in enumerate(correct_indices):
    ax = fig.add_subplot(2, 5, c+1, xticks=[], yticks=[])
    plt.imshow(np.clip(np.transpose(images[idx], (1, 2, 0)), 0, 1))
    ax.set_title(f"{real_labels[int(preds[preds[idx]].cpu().numpy())]}", color=("green" if preds[idx]==labels[idx] else "red"))

# add super title:
fig.suptitle('Correct Predictions', fontsize=20)
plt.show()

In [ ]:
incorrect_indices = []
for i in range(len(labels)):
    if preds[i] != labels[i]:
        incorrect_indices.append(i)

# shuffle the correct indices list:
np.random.shuffle(incorrect_indices)
incorrect_indices = incorrect_indices[:5]

incorrect_indices

In [ ]:
# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(18, 6))

for c, idx in enumerate(incorrect_indices):
    ax = fig.add_subplot(2, 5, c+1, xticks=[], yticks=[])
    plt.imshow(np.clip(np.transpose(images[idx], (1, 2, 0)), 0, 1))
    ax.set_title(f"{real_labels[int(preds[preds[idx]].cpu().numpy())]}", color=("green" if preds[idx]==labels[idx] else "red"))

# add super title:
fig.suptitle('Incorrect Predictions', fontsize=20)
plt.show()

## Question 2

**Task 1:**
- Generate an occlusion attribution heatmap for each image in each set.
- Create a plotting function that generates the data and a 2 x 3 grid of subplots.

In [ ]:
from captum.attr import Occlusion
from captum.attr import FeatureAblation
from captum.attr import visualization as viz
import cv2
from skimage.filters import threshold_otsu

In [ ]:
# Occlusion here
# load all the test set images and labels into memory:
test_images = []
test_labels = []
for images, labels in val_loader:
    test_images.append(images)
    test_labels.append(labels)
test_images = torch.cat(test_images, dim=0)
test_labels = torch.cat(test_labels, dim=0)

IMAGE_INDEX = 11

# Download and uncomment the below to view other random images
# IMAGE_INDEX = np.random.choice(correct_indices)

# Pick image and its label
original_image = test_images[IMAGE_INDEX].unsqueeze(0) # get image
original_label = test_labels[IMAGE_INDEX] # get label 

# move the image and label to the device
original_image = original_image.to(device)
original_label = original_label.to(device)

output = model(original_image)

_, pred_class = torch.max(output, 1) # convert output probabilities to predicted class

**Task 2:**

Repeat the above, but this time use segmentation and feature ablation to explain the predictions.
- Use Otsu's threshold to create a binary image
- Use Connected Components to generate labels

In [ ]:
# Feature segmentation here


In [ ]:
# Feature ablation here

**Task 3**

Extend the segmentation to use LIME analysis. Are there any differences?

In [ ]:
# LIME analysis here